### Check VOMS data: A-30, vehicle inventory, RR-20 data  

This notebook shows the development of the functions that are used in the executable file, `voms_inventory_check.py`

#### Step 1: Compare the A-30 vehicle list to the inventory list in BlackCat
We are replicating the validation steps that the Grants team currently does, in the order that they currently do it, to ensure automating this process is successful. After interviews we know that early on they compare the A-30 vehicle list to the inventory list in BlackCat, and also to the RR-20 reported VOMS vehicles on the RR-20 form.  
  
These are the prototyped checks in this notebook:  
* Check 1. They compare each vehicle's VIN in both places to ensure that all vehicles in the lists are the same.
* Check 2. They make sure that the total "active" vehicles in the inventory are equal to or less than what is reported on the A-30.
* Check 3. They make sure that the total VOMS vehicles reported on the RR-20 is equal to or fewer than the total vehicles on the A-30.
  
The A-30 form is auto-filled with inventory data as it is, *at that time*. It is **not** updated later if the inventory data changes.

---
### TO DO: 
* think through how data will appear (i.e. all orgs in files together, or 1 file per org? Pull in Sharepoint Excel files, or do Sharepoint -> GCS -> python script?), rewrite inputs to functions.
* think through where to output excel files

---


*For now* while deciding on the above, we import excel files of data from i) the current vehicle inventory, ii) 2022 A-30 & iii) RR-20 forms. 
* The vehicle inventory was downloaded from the BlackCat web interface. 
* The A-30 was downloaded from the BlackCat web interface. 
* The RR-20 was downloaded from the BlackCat web interface. ~emailed to us from Black Cat for 2022.~
  
We are assuming that any future API or larger Excel file will have this data in the **exact same format**.  
  
A few data points must match across forms, so we combine those parts of the RR-20 with the accompanying parts of the A-30 into **one data table** upon which to run checks on (columns can be checked against each other).  
  
The rest of the RR-20 and the A-30 will be kept separate to prevent exploding rows by their multiples.


In [1]:
import pandas as pd
import numpy as np 

In [2]:
## Dataset 1. Import the inventory of revenue vehicles from Black Cat.
inventory = pd.read_excel("data/RevenueVehicles_9_2_2023.xlsx", 
                    index_col=None) 

# print(inventory.shape)
# inventory.head(3)

In [3]:
rev_veh_inventory = inventory[['Organization', 'VIN', 'LicensePlate', 'AgencyId', 'Category', 'Status', \
                               'NTD ID', 'Vehicle Type']]

print(rev_veh_inventory.shape)
rev_veh_inventory.head()

(3824, 8)


,Organization,VIN,LicensePlate,AgencyId,Category,Status,NTD ID,Vehicle Type
0,3.00,1FD3E35LX8DB57263,NaN,NaN,Light-Duty Mid-Sized Bus,Active,NaN,CU - Cutaway Bus
1,3.00,1FD3E35LX8DB57264,NaN,NaN,Light-Duty Mid-Sized Bus,Active,NaN,CU - Cutaway Bus
2,Able Industries,1FDEE3FL7FDA08586,43980V1,NaN,Heavy-Duty Small Bus,Active,NaN,CU - Cutaway Bus
3,AbleLight Inc.,1FMNE1BW1BDA36060,6PCV410,A36060,"Light-Duty Vans, Sedans or Buses",Active,NaN,MV - Minivan
4,Alchemia,523MF1A60CM101085,NaN,NaN,"Light-Duty Vans, Sedans or Buses",Active,NaN,BU - Bus


In [4]:
### Dataset 2. Import the a30 and compare.
a30 = pd.read_excel("data/A_30_Revenue_Vehicle_Report_9_1_2023.xlsx", index_col=None)
a30.head(20)

,Organization,Group Plan,VIN,RVI ID,ADA Accessible Vehicles (0/No 1/Yes),Vehicle Type Code,Funding Source,Avg. Estimated Service Years When New,Avg. Expected Service Years When New,Year of Manufacture,Useful Life Remaining,Vehicle Length (ft.),Seating Capacity,Ownership Type,Modes Operated
0,Alpine County Community Development,NaN,1FDES8PG1LKB22727,NaN,Yes,CU - Cutaway Bus,State Aid,10.0,10,2020,5,CU - Cutaway Bus,7.0,Owned outright by public agency (OOPA),DR - Demand Response
1,Alpine County Community Development,NaN,1GB3G2BL5E1199290,NaN,Yes,CU - Cutaway Bus,Section 5311 – Statewide Rural Public Transit,10.0,10,2014,2,CU - Cutaway Bus,8.0,Owned outright by public agency (OOPA),NaN
2,Amador Transit,NaN,2C7WDGBG5GR372437,NaN,Yes,MV - Minivan,Section 5310 – Enhanced Mobility of Seniors & ...,8.0,8,2017,-1,MV - Minivan,4.0,Owned outright by public agency (OOPA),DR - Demand Response
3,Amador Transit,NaN,1FDFE45S29DA54849,NaN,Yes,CU - Cutaway Bus,Section 5317 – New Freedom Program,10.0,10,2009,-6,CU - Cutaway Bus,16.0,Owned outright by public agency (OOPA),"DF - Deviated Fixed Route,DR - Demand Response"
4,Amador Transit,NaN,1FDFE4FS4KDC69024,NaN,Yes,CU - Cutaway Bus,NaN,10.0,10,2019,4,CU - Cutaway Bus,12.0,Owned outright by public agency (OOPA),DF - Deviated Fixed Route
5,Amador Transit,NaN,1FDVU4XG6KKB16858,NaN,Yes,VN - Van,Other Federal Funds: Section 5337- State of Go...,8.0,8,2019,1,VN - Van,8.0,Owned outright by public agency (OOPA),DF - Deviated Fixed Route
6,Amador Transit,NaN,1GB6G5BL0D1113431,NaN,Yes,CU - Cutaway Bus,Section 5310 – Enhanced Mobility of Seniors & ...,10.0,10,2013,-2,CU - Cutaway Bus,16.0,Owned outright by public agency (OOPA),DR - Demand Response
7,Amador Transit,NaN,1GB6G5BL0D1114456,NaN,Yes,CU - Cutaway Bus,Section 5310 – Enhanced Mobility of Seniors & ...,10.0,10,2013,-2,CU - Cutaway Bus,17.0,Owned outright by public agency (OOPA),"DF - Deviated Fixed Route,DR - Demand Response"
8,Amador Transit,NaN,1GB6G5BL2D1113415,NaN,Yes,CU - Cutaway Bus,Section 5310 – Enhanced Mobility of Seniors & ...,10.0,10,2013,-2,CU - Cutaway Bus,16.0,Owned outright by public agency (OOPA),DR - Demand Response
9,Amador Transit,NaN,1GB6G5BL3D1112385,NaN,Yes,CU - Cutaway Bus,Section 5310 – Enhanced Mobility of Seniors & ...,10.0,10,2013,-2,CU - Cutaway Bus,17.0,Owned outright by public agency (OOPA),DF - Deviated Fixed Route


In [5]:
a30.shape

(742, 15)

### Check 1. Compare each vehicle's VIN in both places to ensure that all vehicles in the lists are the same.

*Output option 1: FULL CHECKLIST WITH ALL VINS*  
Plan is to show the grants and 5311 liaision team these output options to get their feedback.

In [10]:
## Inventory: For each agency and year, get all VINS that are active. 
## A30: For each agency and year, get all VINS 
## both:  Compare VIN list with inventory VIN active list
## if matches, spit out totals (e.g., 17 active) and in description col say "all match"
## if does not match, spit out: from inventory and A30, VIN, status. 

inv_agencies = rev_veh_inventory['Organization'].unique()
a30_agencies = a30['Organization'].unique()

output = []
for agency in a30_agencies:
    if len(a30[a30['Organization']==agency]) > 0:
        vins_a30 = a30[a30['Organization']==agency]['VIN'].unique() #will list only active ins
        vins_inventory = rev_veh_inventory[(rev_veh_inventory['Organization']==agency) \
                 & (rev_veh_inventory['Status']=='Active')]['VIN'].unique()
        
        for v in vins_a30: #check whether each VIN exists in inventory
            if v in vins_inventory:
                description = "Matched an Active vehicle in vehicle inventory."
                result = "Y"
            else:
                description = f"{v} not an active vehicle in the inventory. Investigate."
                result = "N"

            output_line = {"Organization": agency,
                           "VIN": v,
                           "check_status": result,
                          "Description": description}
            output.append(output_line)
    
    full_vin_checklist = pd.DataFrame(output).sort_values(by="Organization")


In [11]:
print(full_vin_checklist.shape)
full_vin_checklist

(741, 4)


,Organization,VIN,check_status,Description
0,Alpine County Community Development,1FDES8PG1LKB22727,Y,Matched an Active vehicle in vehicle inventory.
1,Alpine County Community Development,1GB3G2BL5E1199290,N,1GB3G2BL5E1199290 not an active vehicle in the...
18,Amador Transit,4UZADRFDXJCJW7193,Y,Matched an Active vehicle in vehicle inventory.
17,Amador Transit,4UZADRFD8JCJW7192,Y,Matched an Active vehicle in vehicle inventory.
16,Amador Transit,4UZADRFD3JCJW7195,Y,Matched an Active vehicle in vehicle inventory.
...,...,...,...,...
734,Yosemite Area Regional Transportation System,1M8PDMBA5CP01268,Y,Matched an Active vehicle in vehicle inventory.
733,Yosemite Area Regional Transportation System,1M8PDMBA3CP05986,Y,Matched an Active vehicle in vehicle inventory.
732,Yosemite Area Regional Transportation System,1M8PDMBA3CP01268,Y,Matched an Active vehicle in vehicle inventory.
735,Yosemite Area Regional Transportation System,1M8PDMBA5CP05986,Y,Matched an Active vehicle in vehicle inventory.


*OPTION 2: THE OUTPUT IS A LIST OF ONLY THE VINS THAT DON'T MATCH*

In [12]:
inv_agencies = rev_veh_inventory['Organization'].unique()
a30_agencies = a30['Organization'].unique()

output = []
for agency in a30_agencies:
    if len(a30[a30['Organization']==agency]) > 0:
        vins_a30 = a30[a30['Organization']==agency]['VIN'].unique() #will list only active ins
        vins_inventory = rev_veh_inventory[(rev_veh_inventory['Organization']==agency) \
                 & (rev_veh_inventory['Status']=='Active')]['VIN'].unique()
        
        for v in vins_a30: #check whether each VIN exists in inventory
            if v not in vins_inventory:
                description = f"Not an active vehicle in this org's inventory. Investigate."
                result = "N"
                
                output_line = {"Organization": agency,
                           "VIN": v,
                           "check_status": result,
                          "Description": description}
                output.append(output_line)
            else:
                pass
    
    mismatched_vin_checklist = pd.DataFrame(output).sort_values(by="Organization")


In [14]:
## TO DO: add a check for whether it's in the inventory at all with non-active status - if grants team wants it.

print(mismatched_vin_checklist.shape)
mismatched_vin_checklist.head()

(43, 4)


,Organization,VIN,check_status,Description
0,Alpine County Community Development,1GB3G2BL5E1199290,N,Not an active vehicle in this org's inventory....
1,Amador Transit,4UZADRDU4ECFR9869,N,Not an active vehicle in this org's inventory....
2,City of Chowchilla,1FDFE4FS4CDA11717,N,Not an active vehicle in this org's inventory....
11,City of Shafter,3C6TRVPGXFE512763,N,Not an active vehicle in this org's inventory....
10,City of Shafter,3C6TRVPG8FE512762,N,Not an active vehicle in this org's inventory....


---
### Checks 2 & 3: reported totals of active vehicles in both, AND VOMS total from RR-20.  
Output can be that the totals match, or if not, list what each total is.  
  
```Org | n_a30_vehicles | n_rr20_VOMS | n_inventory_vehicles | check_result | notes``` 

In [15]:
# Import relevant part of the rr-20
rr20_service = pd.read_excel("data/NTD_Annual_Report_Rural_2022.xlsx", 
                     sheet_name="Service Data", index_col=None) 


In [16]:
print(rr20_service.shape)
rr20_service.head(5)

(92, 9)


,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0
1,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0
2,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0
3,Amador Transit,NaN,2022,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0
4,Calaveras Transit Agency,CTA,2022,Demand Response (DR) - (PT),23812.0,1416.0,2104.0,0.0,5.0


In [17]:
agency = "Amador Transit"
rr20_service[rr20_service['Organization Legal Name']==agency]['VOMX'].sum()

17.0

In [18]:
inv_agencies = rev_veh_inventory['Organization'].unique()
a30_agencies = a30['Organization'].unique()
rr20_agencies = rr20_service['Organization Legal Name'].unique()

output = []
for agency in a30_agencies:
    if len(a30[a30['Organization']==agency]) > 0:
        a30_n = a30[a30['Organization']==agency]['VIN'].nunique()
        
    if len(rev_veh_inventory[rev_veh_inventory['Organization'] == agency]) > 0:
        inv_n = rev_veh_inventory[(rev_veh_inventory['Organization'] == agency) \
         & (rev_veh_inventory['Status']=='Active')]['VIN'].nunique()
    
    if len(rr20_service[rr20_service['Organization Legal Name']==agency]) > 0:
        rr20_n = rr20_service[rr20_service['Organization Legal Name']==agency]['VOMX'].sum()
        rr20_n = round(rr20_n)

    if (a30_n <= inv_n) & (rr20_n <= inv_n) & (a30_n >= rr20_n):
        result = "pass"
        description = "" #"VOMS & A-30 vehicles reported are equal to and/or lower than active inventory."
    elif (a30_n > inv_n):
        result = "warning"
        description = "More A-30 vehicles reported than in active inventory."
    elif (a30_n < rr20_n):
        result = "fail"
        description = "Total VOMS is greater than total A-30 vehicles reported."

    output_line = {"Organization": agency,
                   "n_a30_vehicles": a30_n,
                   "n_rr20_VOMS": rr20_n,
                   "n_inventory_vehicles": inv_n,
                   "check_result": result,
                   "Description": description}
    output.append(output_line)
    totals_checklist = pd.DataFrame(output)


In [19]:
print(totals_checklist.shape)
totals_checklist

(48, 6)


,Organization,n_a30_vehicles,n_rr20_VOMS,n_inventory_vehicles,check_result,Description
0,Alpine County Community Development,2,1,1,fail,More A-30 vehicles reported than in active inv...
1,Amador Transit,17,17,19,pass,
2,Calaveras Transit Agency,10,11,10,fail,Total VOMS is greater than total A-30 vehicles...
3,City of Arcata,6,2,6,pass,
4,City of Arvin,6,4,7,pass,
5,City of Auburn,7,6,7,pass,
6,City of Chowchilla,3,2,2,fail,More A-30 vehicles reported than in active inv...
7,City of Corcoran - Corcoran Area Transit,6,6,6,pass,
8,City of Dixon,12,10,12,pass,
9,City of Escalon,3,2,3,pass,


---
### Export all draft outputs to excel files
We also add a few more columns for Liaisions to manually track agency responses.  
Again, plan is to show the grants and 5311 liaision team these output options to get their feedback.

In [29]:
# create a excel writer object
with pd.ExcelWriter("reports/voms_check_report.xlsx") as writer:
   
    full_vin_checklist.to_excel(writer, sheet_name="vin_check_full", index=False, startrow=2)
    mismatched_vin_checklist.to_excel(writer, sheet_name="vin_check_fails_only", index=False, startrow=2)
    totals_checklist.to_excel(writer, sheet_name="totals_check", index=False, startrow=2)
    
    workbook = writer.book
    worksheet1 = writer.sheets["vin_check_full"]
    worksheet2 = writer.sheets["vin_check_fails_only"]
    worksheet3 = writer.sheets["totals_check"]
    cell_highlight = workbook.add_format({
        'fg_color': 'yellow',
        'bold': True,
        'border': 1
    })
    
    report_title = "NTD Data Validation Report"
    title_format = workbook.add_format({
              'bold': True,
              'valign': 'center',
              'align': 'left',
              'font_color': '#1c639e',
              'font_size': 15
            })
    subtitle1 = "VOMS Inventory Vehicle Check: Validation Warnings"
    subtitle2 = "VOMS RR-20 & A-30 check"
    subtitle_format = workbook.add_format({
          'bold': True,
          'align': 'left',
          'font_color': 'black',
          'font_size': 19
        })

    # Column width and format - sheet 1
    worksheet1.set_column(0, 0, 35) #col A width
    worksheet1.set_column(1, 2, 20) #cols B-C width
    worksheet1.set_column(3, 3, 53) #col D width
    worksheet1.write('A1', report_title, title_format)
    worksheet1.write('A2', subtitle1, subtitle_format)
    worksheet1.freeze_panes('B4')
    
    # Column width and format - sheet 2
    worksheet2.set_column(0, 0, 35) #col A width
    worksheet2.set_column(1, 2, 20) #cols B-C width
    worksheet2.set_column(3, 5, 53) #cols D-F width
    worksheet2.write('E3', 'Agency Response', cell_highlight)
    worksheet2.write('F3', 'Response Date', cell_highlight)
    worksheet2.write('A1', report_title, title_format)
    worksheet2.write('A2', subtitle1, subtitle_format)
    worksheet2.freeze_panes('B4')

    
    # Column width and format - sheet 3
    worksheet3.set_column(0, 0, 35) #col A width
    worksheet3.set_column(1, 4, 18) #cols B-E width
    worksheet3.set_column(5, 7, 53) #cols F-H width
    worksheet3.write('G3', 'Agency Response', cell_highlight)
    worksheet3.write('H3', 'Response Date', cell_highlight)
    worksheet3.write('A1', report_title, title_format)
    worksheet3.write('A2', subtitle2, subtitle_format)
    worksheet3.freeze_panes('B4')